<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [2]:
df=pd.read_csv("/content/Churn_Modelling.csv")
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
df.describe().T

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.shape

In [ ]:
df.drop(["RowNumber","CustomerId","Surname"],axis=1 ,inplace=True)

In [ ]:
label=LabelEncoder()
df["Gender"]=label.fit_transform(df["Gender"])

In [ ]:
ohe=OneHotEncoder(sparse_output=False)
ohe_geo=ohe.fit_transform(df[["Geography"]])

In [ ]:
geo=pd.DataFrame(ohe_geo,columns=ohe.get_feature_names_out())

In [ ]:
df=pd.concat([df.drop("Geography",axis=1),geo],axis=1)

In [ ]:
df.head()

In [ ]:
with open ("label.pkl","wb") as file:
  pickle.dump(label,file)

In [ ]:
with open ("ohe.pkl","wb") as file:
  pickle.dump(ohe,file)

In [ ]:
X=df.drop("Exited",axis=1)
Y=df["Exited"]

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
scaller=StandardScaler()
X_train=scaller.fit_transform(X_train)
X_test=scaller.fit_transform(X_test)

In [ ]:
with open ("scaller.pkl",'wb') as file:
  pickle.dump(scaller,file)

# ANN

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
import datetime


In [ ]:
## building ann model
model=Sequential([
    Dense(64,activation="relu",input_shape=(X_train.shape[1],)),#fist hidden layer connected to the input layer
    Dense(32,activation="relu"),#hl2
    Dense(1,activation="sigmoid")#output layer

])

In [ ]:
model.summary()

In [ ]:
## compile this model
model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              metrics=['accuracy'])

In [ ]:
## seting tensor borad
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="log/fit/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_calllback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
# seting early stoping
early_stopping=EarlyStopping(monitor="val_loss",patience=20,restore_best_weights=True)


In [ ]:
# training the model
history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=100,
                  callbacks=[tensorflow_calllback,early_stopping])

In [ ]:
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
Y_pred = model.predict(X_test)
Y_pred_classes = (Y_pred > 0.5).astype("int64")
Y_pred=model.predict(X_test)


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


# Generate classification report
print("\nClassification Report:")
print(classification_report(Y_test,Y_pred_classes ))

# Compute confusion matrix
conf_matrix = confusion_matrix(Y_test, Y_pred_classes)
print("\nConfusion Matrix:")
print(conf_matrix)

# Visualize confusion matrix (optional)
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
model.save("model.h5")

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir log/

#How to load pickel file


In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
# load the trained model ,scaler pickle, onehot
model=load_model("model.h5")
with open ("label.pkl",'rb') as file:
  label_gender=pickle.load(file)

with open("ohe.pkl",'rb') as file:
  label_geo=pickle.load(file)
with open ("scaller.pkl",'rb')as file:
  scaller=pickle.load(file)


In [ ]:
inputdata={
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    'HasCrCard':1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}


In [ ]:
#one hot encoding
geo_encode=label_geo.transform([[inputdata["Geography"]]])
geo_encode_df=pd.DataFrame(geo_encode,columns=label_geo.get_feature_names_out())
geo_encode_df

In [ ]:
input_df=pd.DataFrame([inputdata])
input_df

In [ ]:
#encode cat variable
input_df["Gender"]=label_gender.transform(input_df["Gender"])

In [ ]:
#concate the gender and geography data (one hot encoding)
input_df=pd.concat([input_df.drop("Geography",axis=1),geo_encode_df],axis=1)

In [ ]:
#scalling the input
input_scaller=scaller.transform(input_df)
input_scaller

In [ ]:
prediction=model.predict(input_scaller)
prediction

In [ ]:
prediction_proba=prediction[0][0]
prediction_proba

In [ ]:
if prediction_prob>0.5:
  print("customer is likely bhagoda")
else:
  print("customer is not likely bhagoda")